In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from skportfolio.frontier._mad import EfficientMeanAbsoluteDeviation

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pypfopt.expected_returns import prices_from_returns, returns_from_prices
from skportfolio.data import load_tech_stock_prices
from skportfolio import MeanHistoricalLinearReturns, SampleCovariance, EquallyWeighted

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_context("notebook")
sns.set(font="Nimbus Sans")
plt.style.use('ggplot')
sns.set_style('ticks')
%config InlineBackend.figure_format="svg"
mpl.rcParams["figure.autolayout"] = True

mpl.rcParams['font.family'] = 'Arial'


In [ ]:
m = np.array([ 0.05,  0.1,  0.12,  0.18 ])
C = np.array([[ 0.0064, 0.00408, 0.00192, 0,], [0.00408, 0.0289, 0.0204, 0.0119], [0.00192, 0.0204, 0.0576, 0.0336], [0, 0.0119, 0.0336, 0.1225,]])
m = m/12
C = C/12

r = np.random.multivariate_normal(mean=m,cov=C, size=20000)
np.savetxt('matlab/mad/returns.txt', r)
returns = pd.DataFrame(r, columns=['A','B','C','D'])
prices = prices_from_returns(returns)

In [ ]:
mu = returns.mean()

## Calculation of efficient frontier

In [ ]:
w_min = pd.Series(
    EfficientMeanAbsoluteDeviation(
        expected_returns=mu,
        returns=returns
    ).min_risk(),
    index=returns.columns
)
w_min

In [ ]:
%%timeit
N=10

w_min = pd.Series(
    EfficientMeanAbsoluteDeviation(
        expected_returns=mu,
        returns=returns
    ).min_risk(),
    index=returns.columns
)
min_risk = np.mean(np.abs(w_min@(returns - mu).T))
min_ret = w_min.T @ mu
rets = np.linspace(min_ret, np.max(mu)-1E-6, N)
risks = np.array([np.nan]*N)
all_weights = []
for i,r in enumerate(rets):
    try:
        MAD = EfficientMeanAbsoluteDeviation(
            expected_returns=mu,
            returns=returns
        )
        weights = pd.Series(
            MAD.efficient_return(target_return=r),
            index=returns.columns
        )
        risks[i] = np.mean(np.abs(weights@(returns-mu).T))
        all_weights.append(weights)
    except Exception as ex:
        print(ex)

risks = np.array(risks)
all_weights = np.array(all_weights)

In [ ]:
all_weights.round(4)

In [ ]:
M = pd.DataFrame(np.loadtxt('matlab/mad/matlab_weights.txt',delimiter=',')).assign(origin='matlab')

SK=pd.DataFrame(all_weights).assign(origin='skptf')

In [ ]:
pd.DataFrame(
    (
        (
            (
                np.loadtxt('matlab/mad/matlab_weights.txt',delimiter=',') - all_weights)
        )
    )
).mul(10000).plot.bar(
    ylabel='$\Delta$ BPS',
    xlabel='portfolio',
    title='Matlab vs scikit-portfolio'
)
plt.grid()
plt.savefig('docs/imgs/mad_matlab_vs_skportfolio.svg', bbox_inches='tight')

In [ ]:
plt.plot(risks,rets,'gray')
plt.plot(risks,rets,'^',label='scikit-portfolio')
plt.plot(
    [
        np.mean(np.abs(w@(returns-mu).T)) for w in np.loadtxt('matlab/mad/matlab_weights.txt',delimiter=',')
    ],
    [
        w@mu for w in np.loadtxt('matlab/mad/matlab_weights.txt',delimiter=',')
    ],
    'v',
    label='Matlab'
)
plt.legend()
plt.title('Efficient frontier')
plt.xlabel('Risk (MAD)')
plt.ylabel('Return')
for i,(ri,re) in enumerate(zip(risks[1:],rets[1:])):
    plt.text(ri+0.0001, re-0.0008,str(i+1))
plt.text(risks[0]+0.001,rets[0]-0.0002,'0')
plt.grid()
plt.savefig('docs/imgs/mad_matlab_efficient_frontier.svg',bbox_inches='tight')

In [ ]:
## efficient risk
w_eff_risk = pd.Series(
    EfficientMeanAbsoluteDeviation(
        expected_returns=mu,
        returns=returns
    ).efficient_risk(target_risk=0.035),
    index=returns.columns
)
eff_risk = 0.035#np.mean(np.abs(w_eff_risk@(returns - mu).T))
eff_ret = w_eff_risk.T@mu

## efficient return
w_eff_ret = pd.Series(
    EfficientMeanAbsoluteDeviation(
        expected_returns=mu,
        returns=returns
    ).efficient_return(target_return=0.011),
    index=returns.columns
)
eff_ret2 = np.mean(np.abs(w_eff_ret@(returns - mu).T))
eff_risk2 = w_eff_ret.T@mu

In [ ]:
fig, ax = plt.subplots()
plt.plot(risks,rets)
ax.set_xlabel('L1-risk')
ax.set_ylabel('Return')
ax.grid(True)
ax.set_title('MAD frontier', fontsize=14)
ax.axvline(x=risks[25], ymin=0, ymax=0.5, color='purple', linestyle='solid')
ax.axhline(y=rets[25], xmin=0, xmax=0.35, color='purple', linestyle='dashed')

ax.axvline(x=risks[35], ymin=0, ymax=0.7, color='darkgreen', linestyle='dashed')
ax.axhline(y=rets[35], xmin=0, xmax=0.55, color='darkgreen', linestyle='solid')

ax.scatter(x=np.mean(np.abs((mu==np.max(mu)).astype(float)@(returns - mu).T)), y=np.max(mu), color='black')
ax.scatter(x=min_risk, y=min_ret, color='red')

ax.scatter(x=risks[35],y=rets[35],color='darkgreen')
ax.scatter(x=risks[25],y=rets[25],color='purple')
ax.annotate('target return',xy=(0.03,0.0115),color='darkgreen')

ax.annotate('target risk',xy=(0.04,0.005),color='purple',rotation=90)
plt.tight_layout()
plt.savefig('docs/imgs/mad_efficient_frontier.svg',dpi=200, bbox_inches='tight')

In [ ]:
model = EfficientAbsoluteDeviation(
    expected_returns = returns.mean(),
    returns = returns,
    risk_free_rate = 0.0,
)
model.minimum_l1_risk()
l1_risk = model._opt.value
# model.efficient_risk(target_risk=0.03756750624124884)
# model._opt.value

In [ ]:
model = EfficientAbsoluteDeviation(
    expected_returns = returns.mean(),
    returns = returns,
    risk_free_rate = 0.0,
)
model.efficient_risk(target_risk=l1_risk)
ret_at_target_risk = model._opt.value

In [ ]:
model = EfficientAbsoluteDeviation(
    expected_returns = returns.mean(),
    returns = returns,
    risk_free_rate = 0.006,
)
model.tangency()

In [ ]:
model._opt.value

In [ ]:
np.mean(np.abs((returns-m)@model.weights)-0.006)

In [ ]:
m.dot(model.weights)

In [ ]:
from skportfolio._omega import EfficientOmegaRatio

In [ ]:
model = EfficientOmegaRatio(m,returns)#.max_omega_ratio()
model.max_omega_ratio()

In [ ]:
model.portfolio_performance()

In [ ]:
model.weights.dot(m*12)

In [ ]:
m.

In [ ]:
EfficientAbsoluteDeviation(
    expected_returns = returns.mean(),
    returns = returns,
    risk_free_rate = 0.0,
).minimum_l1_risk()